# Preamble: The uninteresting but necessary functions

In [ ]:
##### Ignore this ####
#We test it on the feasible region ```feas_regs[0]``` and two of the vertices of the corresponding polytron.
'''
a,b,c,d,e,f,x,y = var('a,b,c,d,e,f,x,y')
region = [[0, -1, 1, 0, 0, 0, 1],
         [0, 0, 0, -1, 1, 1, 0],
         [0, 1, 0, 0, 0, 1, -1],
         [0, 0, 0, 1, 0, -1, 1],
         [0, 1, -1, 0, 1, 0, 0],
         [0, 0, 1, 1, -1, 0, 0]]
v1 = [a, b]
v2 = [-e + f, -e]
points = [v1, v2]
'''

In [1]:
import re
def symbols_to_vector(symbolic_expression):
    symbols_vec = [0,a,b,c,d,x,y]
    string = str(symbolic_expression)
    vec = [0,0,0,0,0,0,0]
    sign = 1
    if string == "0":
        return vector(vec)
    for letter in string:
        if letter in [" "]:
            continue
        elif letter in ["+"]:
            sign = 1
        elif letter in ["-"]:
            sign = -1
        elif var(letter) in symbols_vec:
            index = symbols_vec.index(var(letter))
            vec[index] = 1*sign
    return vector(vec)

def vector_to_symbols(vect):
    symbols_vec = vector([0,a,b,c,d,x,y])
    return symbols_vec.dot_product(vector(vect))

def string_to_symbols(string):
    #input string of the form a + b - c
    #output: symbolic expresseion a+b-c
    if string[0] in ['-']:
        sign = -1
        string = string[1:]
    else:
        sign = 1
    res = 0
    spl = string.split()
    for letter in spl:
        if letter in ['+']:
            sign = 1
        elif letter in ['-']:
            sign = -1
        else:
            symbol = var(letter)
            res += sign*symbol
    return res

In [2]:
class tropical_line: #min tropical line
    def __init__(self, apex, assumptions):
        self.apex = apex
        self.terms = [x-apex[0],y-apex[1],0]
        self.assumptions = assumptions
        self.horizontal = self.get_horizontal()
        self.vertical = self.get_vertical()
        self.diagonal = self.get_diagonal()
        self.halfrays = [self.vertical, self.horizontal, self.diagonal]
        
    def get_diagonal(self):
        vectors = [symbols_to_vector(term) for term in self.terms]
        equations = [vectors[0]-vectors[1]]
        inequalities = [vectors[2]-vectors[0],vectors[2]-vectors[1]]
        return Polyhedron(eqns = equations, ieqs = inequalities+self.assumptions)

    def get_horizontal(self):
        vectors = [symbols_to_vector(term) for term in self.terms]
        equations = [vectors[1]-vectors[2]]
        inequalities = [vectors[0]-vectors[1],vectors[0]-vectors[2]]
        return Polyhedron(eqns = equations, ieqs = inequalities+self.assumptions)

    def get_vertical(self):
        vectors = [symbols_to_vector(term) for term in self.terms]
        equations = [vectors[0]-vectors[2]]
        inequalities = [vectors[1]-vectors[0],vectors[1]-vectors[2]]
        return Polyhedron(eqns = equations, ieqs = inequalities+self.assumptions)

In [6]:
a, b, c, d, x, y = var('a, b, c, d, x, y')

# The Interesting Part

We consider the unique tropical lines (min) that has the point $(a,b)$ as apex. That is, we consider the tropical variety defined by $f(x,y) = \text{min} \{x-a, y-b, 0 \}$. Each of the halfrays of the tropical line are given by a set of equations and inequalities. We refer to these halfrays as ```horizontal, vertical``` and ```diagonal```.

It consists of three halfrays: The ```horizontal``` halfray is given by the set of $x-a = 0 \leq y-b$, the halfrays ```vertical``` and ```diagonal``` are defined analogously.

A tropical line always carries global ```assumptions``` about the relationship between the variables with it, e.g. $a>c, d<b$.

If the assumptions come from a cone in the Kleene star fan, the assumptions look like this:

In [7]:
region = [[0,1,0,-1,0],  #c<a
          [0,0,1,0,-1],  #d<b
          [0,1,-1,-1,1]  #c-a<d-b
         ]

Since the inequalities in the region do not take the variables $x$ and $y$ into account, we have to extend these vectors in order to embed them into $\mathbb{R}^6$.

In [8]:
global_assumptions = [el+[0,0] for el in region]
global_assumptions

[[0, 1, 0, -1, 0, 0, 0], [0, 0, 1, 0, -1, 0, 0], [0, 1, -1, -1, 1, 0, 0]]

Let's create a tropical line that has apex $(a,b)$

In [9]:
t1 = tropical_line(apex = [a,b], assumptions = global_assumptions)
t1.terms

[-a + x, -b + y, 0]

The ```vertical``` halfray of ```t1``` is a polyhedron defined by the inequalities $x-a = 0 \leq y-b$ and the ```assumptions``` $c<a, d<b, c-a<d-b$.

In [10]:
t1.vertical.Hrepresentation()

(An equation (1, 0, 0, 0, -1, 0) x + 0 == 0,
 An inequality (0, -1, 0, 0, 0, 1) x + 0 >= 0,
 An inequality (0, 1, 0, -1, 0, 0) x + 0 >= 0,
 An inequality (1, -1, -1, 1, 0, 0) x + 0 >= 0)

Let's define a tropical line ```t2``` with apex $(c,d)$ and compute the intersection point of ```t1.diagonal``` and ```t2.horizontal``` under the ```assumptions``` $c<a, d<b, c-a<d-b$.

In [11]:
t2 = tropical_line(apex = [c,d], assumptions = global_assumptions)
t2.terms

[-c + x, -d + y, 0]

In [12]:
intersection = t1.diagonal.intersection(t2.horizontal)
intersection.Hrepresentation()

(An equation (0, 0, 0, 1, 0, -1) x + 0 == 0,
 An equation (1, -1, 0, 1, -1, 0) x + 0 == 0,
 An inequality (1, -1, -1, 1, 0, 0) x + 0 >= 0,
 An inequality (0, 1, 0, -1, 0, 0) x + 0 >= 0)

Translate these equations and inequalities to symbolic expressressions:

In [13]:
equations    = [vector_to_symbols([0]+list(eqn.A())) ==0 for eqn in intersection.equations()]
equations

[d - y == 0, a - b + d - x == 0]

In [14]:
inequalities = [vector_to_symbols([0]+list(ieq.A())) > 0 for ieq in intersection.inequalities()]
inequalities

[a - b - c + d > 0, b - d > 0]

Solve this system of inequalities / equations for x and y!

In [15]:
solution = solve(inequalities+equations, x,y)
solution

[[x == a - b + d, y == d, b - d > 0, a - b - c + d > 0]]

Extract the relevant data from this solution:

In [17]:
#parsing the solution
relevant = [re.split(' == ', str(el)) for el in solution[0] if ' == ' in str(el)]
res = [None,None]
for el in relevant:
    if 'x' in el:
        el.remove('x')
        res[0] = string_to_symbols(el[0])
    elif 'y' in el:
        el.remove('y')
        res[1] = string_to_symbols(el[0])

print 'intersection point: '+str(res)

intersection point: [a - b + d, d]


If we instead consider the intersection of ```t1.diagonal``` and ```t2.vertical``` under the ```assumptions``` $c<a, d<b, c-a<d-b$, we have the equation ```(1, -1, -1, 1, 0, 0) x + 0 == 0```, which translates to $a-c = d-b$.

In [18]:
intersection = t1.diagonal.intersection(t2.vertical)
intersection.Hrepresentation()

(An equation (1, -1, -1, 0, 0, 1) x + 0 == 0,
 An equation (0, 0, 1, 0, -1, 0) x + 0 == 0,
 An equation (1, -1, -1, 1, 0, 0) x + 0 == 0,
 An inequality (1, 0, -1, 0, 0, 0) x + 0 >= 0)

This means that all solutions in this (closed) polyhedron satisfy $c-a\leq d-b$ only with equality. In this case, we can say that the $2$-dimensional halfrays do not intersect, given that our assumptions containing only strict inequalities. 

Compare each pair of halfrays to get the intersection point:

In [19]:
res = [0,0]
count = 0
for ray1 in t1.halfrays:
    for ray2 in t2.halfrays:
        intersection = ray1.intersection(ray2)
        eq = [list(equation.A()) for equation in intersection.equations()]
        if [1, 0, -1, 0, 0, 0] in eq or [0, 1, 0, -1, 0, 0] in eq or [1,-1,-1,1,0,0] in eq:
            continue
        else:
            inequalities = [vector_to_symbols([0]+list(ieq.A())) > 0 for ieq in intersection.inequalities()]
            equations    = [vector_to_symbols([0]+list(eqn.A())) ==0 for eqn in intersection.equations()]
            solution = solve(inequalities+equations, x,y)
            if len(solution)==1:
                #parsing the solution
                relevant = [re.split(' == ', str(el)) for el in solution[0] if ' == ' in str(el)]
                for el in relevant:
                    if 'x' in el:
                        el.remove('x')
                        res[0] = string_to_symbols(el[0])
                    elif 'y' in el:
                        el.remove('y')
                        res[1] = string_to_symbols(el[0])
                print res
            else:
                print 'more than one solution'

[a - b + d, d]
